In [ ]:
import plotly.plotly as py
import pandas as pd
import numpy as np
from loclust import parse
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()
from teslaver import utilities as utls
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF

parsed_dataframe = utls.trajectories_to_dataframe(parse.read_trajectories('../data/CO2_trajs_short.csv'))
years = [traj.time for traj in parse.read_trajectories('../data/CO2_trajs_short.csv')]
figure = {
        'data': [],
        'layout': {},
        'frames': [],   
        'config': {'scrollzoom': True}
    }
figure['layout']['xaxis'] = {'title': 'Year', 'autorange':True}
figure['layout']['yaxis'] = {'title': 'Kg of Carbon Emissions Per Dollar of GDP per Capita', 'autorange':True}
figure['layout']['hovermode'] = 'closest'
figure['layout']['showlegend'] = True
figure['layout']['sliders'] = {
    'args':[
        'slider.value', {
            'duration': 400,
            'ease': 'cubic-in-out'
        }
    ],
    'initialValue': '1990',
    'plotlycommand': years,
    'visible':True
}

figure['layout']['updatemenus'] = [
    {
        'buttons':[
            {
                'args':[None, {'frame':{'duration': 500, 'redraw': False}}],
                'label':'Play',
                'method':'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad':{'r':10, 't':87},
        'showactive':False,
        'type':'buttons',
        'x':0.1,
        'xanchor':'right',
        'y':0,
        'yanchor':'top',
    }
]

sliders_dict = {
    'active':0,
    'yanchor':'top',
    'xanchor':'left',
    'currentvalue':{
        'font' : {'size': 20},
        'prefix':'Year',
        'visible':True,
        'xanchor':'right'
    },
    'transition':{'duration':300, 'easing':'cubic-in-out'},
    'pad':{'b':10, 't':50},
    'len':0.9,
    'x':0.1,
    'y':0,
    'steps':[]
}

custom_colors = {
    'Asia': 'rgb(171, 99, 250)',
    'Europe': 'rgb(230, 99, 250)',
    'Africa': 'rgb(99, 110, 250)',
    'Americas': 'rgb(25, 211, 243)',
    'Oceania': 'rgb(50, 170, 255)'
}

grouped_df = parsed_dataframe.groupby('X')
for year in grouped_df.groups.keys():
    frame = {'data': [], 'name': str(year)}
    for index, row in grouped_df.get_group(year).iterrows():
        datadict = {
            'x' : year,
            'y' : row['Y'],
            'mode': 'markers',
            'text': row['country_name'],
            'marker': {
                'sizemode': 'area',
                'sizeref': 200000,
                'size': 30
            }
        }
        frame['data'].append(datadict)
    figure['frames'].append(frame) 
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
    
figure['layout']['sliders'] = [sliders_dict]
iplot(figure, 'C02Graph')